In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import networkx as nx
import csv
import numpy as np
import sklearn
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Data cleaning

In [ ]:
years = [year for year in range(2021, 2009, -1)]
#years =['2011', '2010']

In [ ]:
import re

def process_excel_file(year, excel_path):
    df = pd.read_excel(excel_path, sheet_name=1, header=None)


    df = df.transpose().reset_index()
    df.columns = df.iloc[0]
    # Drop the first row
    df = df.iloc[1:]

    df.columns = ['Owner_occupied_' + col if 10>= i > 4 else col for i, col in enumerate(df.columns)]
    df.columns = ['Renter_occupied_' + col if 17>= i > 11 else col for i, col in enumerate(df.columns)]
    df = df.rename(columns={'Owner occupied:' : 'Owner_occupied', 'Renter occupied:' : 'Renter_occupied'})

    new_column_name = 'census_tract'  # Replace with the new name
    df.columns.values[1] = new_column_name
    df = df.dropna(subset=['census_tract'])
    #df['census_tract'] = df['census_tract'].apply(lambda x: re.sub(r'[^0-9.]', '', x) if isinstance(x, str) else x)
    df['census_tract'] = df['census_tract'].apply(lambda x: re.sub(r'[^0-9]', '', x) if isinstance(x, str) else x)
    df = df.reset_index()
    df = df.drop(columns = ['Label', 'index', 0], axis=1)
    df['year'] = year




    return df

In [ ]:
for year in years:
  excel_file_path = f'/content/drive/MyDrive/DS project/Jamaica_Plain_homeownership/ACSDT5Y{year}.xlsx'
  variable_name = f'Jamaica_Plain_homeownership_{year}'

        # Call the function and assign the result to the variable
  globals()[variable_name] = process_excel_file(year, excel_file_path)

In [ ]:
for year in years:
  df = globals()[f'Jamaica_Plain_homeownership_{year}']
  for column in df.columns[: -1]:
    df[column] = df[column].str.replace('[^0-9-]', '', regex=True).astype(int)

In [ ]:
for df in [Jamaica_Plain_homeownership_2021, Jamaica_Plain_homeownership_2020, Jamaica_Plain_homeownership_2019, Jamaica_Plain_homeownership_2018]:

  df['Owner_occupied_Moved in 1999 or earlier'] =  df.iloc[:, 7:9].sum(axis=1)
  df['Renter_occupied_Moved in 1999 or earlier'] = df.iloc[:, 14:16].sum(axis=1)

In [ ]:
for df in [Jamaica_Plain_homeownership_2017, Jamaica_Plain_homeownership_2016, Jamaica_Plain_homeownership_2015]:
  df['Owner_occupied_Moved in 1999 or earlier'] =  df.iloc[:, 6:9].sum(axis=1)
  df['Renter_occupied_Moved in 1999 or earlier'] = df.iloc[:, 13:16].sum(axis=1)

In [ ]:
for df in [Jamaica_Plain_homeownership_2014, Jamaica_Plain_homeownership_2013, Jamaica_Plain_homeownership_2012, Jamaica_Plain_homeownership_2011, Jamaica_Plain_homeownership_2010]:
  df['Owner_occupied_Moved in 1999 or earlier'] =  df.iloc[:, 5:9].sum(axis=1)
  df['Renter_occupied_Moved in 1999 or earlier'] = df.iloc[:, 12:16].sum(axis=1)

In [ ]:
Jamaica_Plain_homeownership_2014.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 19 columns):
 #   Column                                       Non-Null Count  Dtype
---  ------                                       --------------  -----
 0   census_tract                                 3 non-null      int64
 1   Total population in occupied housing units:  3 non-null      int64
 2   Owner_occupied                               3 non-null      int64
 3   Owner_occupied_Moved in 2010 or later        3 non-null      int64
 4   Owner_occupied_Moved in 2000 to 2009         3 non-null      int64
 5   Owner_occupied_Moved in 1990 to 1999         3 non-null      int64
 6   Owner_occupied_Moved in 1980 to 1989         3 non-null      int64
 7   Owner_occupied_Moved in 1970 to 1979         3 non-null      int64
 8   Owner_occupied_Moved in 1969 or earlier      3 non-null      int64
 9   Renter_occupied                              3 non-null      int64
 10  Renter_occupied_Moved in 2010 

In [ ]:
#list_of_indices
# Create a list of DataFrames
concatenated_df = [globals()[f'Jamaica_Plain_homeownership_{year}'] for year in years]

# Initialize an empty DataFrame to store the concatenated result
result_df2 = pd.DataFrame()

# Concatenate the DataFrames in the list
for df in concatenated_df:
    result_df2 = pd.concat([result_df2, df], ignore_index=True)

In [ ]:
result_df2.columns

Index(['census_tract', 'Total population in occupied housing units:',
       'Owner_occupied', 'Owner_occupied_Moved in 2019 or later',
       'Owner_occupied_Moved in 2015 to 2018',
       'Owner_occupied_Moved in 2010 to 2014',
       'Owner_occupied_Moved in 2000 to 2009',
       'Owner_occupied_Moved in 1990 to 1999',
       'Owner_occupied_Moved in 1989 or earlier', 'Renter_occupied',
       'Renter_occupied_Moved in 2019 or later',
       'Renter_occupied_Moved in 2015 to 2018',
       'Renter_occupied_Moved in 2010 to 2014',
       'Renter_occupied_Moved in 2000 to 2009',
       'Renter_occupied_Moved in 1990 to 1999',
       'Renter_occupied_Moved in 1989 or earlier', 'year',
       'Owner_occupied_Moved in 1999 or earlier',
       'Renter_occupied_Moved in 1999 or earlier',
       'Owner_occupied_Moved in 2017 or later',
       'Owner_occupied_Moved in 2015 to 2016',
       'Renter_occupied_Moved in 2017 or later',
       'Renter_occupied_Moved in 2015 to 2016',
       'Owner_

In [ ]:
result_df3 = result_df2[['census_tract', 'Total population in occupied housing units:',
       'Owner_occupied', 'Owner_occupied_Moved in 1999 or earlier',
      'Renter_occupied', 'Renter_occupied_Moved in 1999 or earlier',
       'year']]

In [ ]:
result_df3.sort_values(by=['census_tract', 'year'], inplace=True)

result_df3['percent_owners'] = result_df3['Owner_occupied']/result_df3['Total population in occupied housing units:']
result_df3['percent_renters'] = result_df3['Renter_occupied']/result_df3['Total population in occupied housing units:']

result_df3['percent_old_owners'] = result_df3['Owner_occupied_Moved in 1999 or earlier']/result_df3['Owner_occupied']
result_df3['percent_old_renters'] = result_df3['Owner_occupied_Moved in 1999 or earlier']/result_df3['Renter_occupied']

result_df3['renters_owners_ratio'] = result_df3['Owner_occupied']/result_df3['Renter_occupied']
result_df3['old_renters_owners_ratio'] = result_df3['Owner_occupied_Moved in 1999 or earlier']/result_df3['Renter_occupied_Moved in 1999 or earlier']

# Calculate the percentage change within each 'census_tract' group
#result_df3['percentage_change'] = result_df3.groupby('census_tract')['Owner_occupied_Moved in 1999 or earlier'].pct_change()

<ipython-input-16-236f53da9bbd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df3.sort_values(by=['census_tract', 'year'], inplace=True)
<ipython-input-16-236f53da9bbd>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df3['percent_owners'] = result_df3['Owner_occupied']/result_df3['Total population in occupied housing units:']
<ipython-input-16-236f53da9bbd>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

In [ ]:
result_df3['census_tract'] = result_df3['census_tract'].astype('float64')

# Adding back the decimal to census tract..
def custom_replace(value):
    if value > 10000:
        return value/100
   # else: return int(value)

    return value


result_df3['census_tract'] = result_df3['census_tract'].apply(custom_replace)

<ipython-input-17-1fae1e3157fa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df3['census_tract'] = result_df3['census_tract'].astype('float64')
<ipython-input-17-1fae1e3157fa>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df3['census_tract'] = result_df3['census_tract'].apply(custom_replace)


In [ ]:
result_df3.reset_index(drop=True)

,census_tract,Total population in occupied housing units:,Owner_occupied,Owner_occupied_Moved in 1999 or earlier,Renter_occupied,Renter_occupied_Moved in 1999 or earlier,year,percent_owners,percent_renters,percent_old_owners,percent_old_renters,renters_owners_ratio,old_renters_owners_ratio
0,1101.03,5742,2093,566,3649,395,2010,0.364507,0.635493,0.270425,0.155111,0.573582,1.432911
1,1101.03,5859,2228,738,3631,281,2011,0.380270,0.619730,0.331239,0.203250,0.613605,2.626335
2,1101.03,5888,2569,918,3319,258,2012,0.436311,0.563689,0.357337,0.276589,0.774028,3.558140
3,1101.03,5849,2634,1123,3215,173,2013,0.450333,0.549667,0.426348,0.349300,0.819285,6.491329
4,1101.03,6218,3236,1679,2982,275,2014,0.520425,0.479575,0.518850,0.563045,1.085178,6.105455
5,1101.03,6548,2911,1641,3637,260,2015,0.444563,0.555437,0.563724,0.451196,0.800385,6.311538
6,1101.03,6538,3248,1618,3290,341,2016,0.496788,0.503212,0.498153,0.491793,0.987234,4.744868
7,1101.03,6293,3135,1374,3158,288,2017,0.498173,0.501827,0.438278,0.435085,0.992717,4.770833
8,1101.03,6657,3327,1728,3330,344,2018,0.499775,0.500225,0.519387,0.518919,0.999099,5.023256
9,1101.03,6803,3303,1638,3500,152,2019,0.485521,0.514479,0.495913,0.468000,0.943714,10.776316


In [ ]:
result_df3.to_csv('/content/drive/MyDrive/DS project/Jamaica_Plain_homeownership/Jamaica_Plain_houseownership.csv')